# Metabolites

## Description
In this notebook we create a dataframe containing all the available information for the metabolites accounted in our reconstruction.

In [1]:
# Import libraries
import gspread
import pandas as pd
import numpy as np
import cobra
from cobra.io import read_sbml_model

from tqdm.notebook import tqdm

/Users/pablodigiusto/miniforge3/envs/mncho/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# Define functions

def df_to_dict(df, key_col):
    """
    This function takes a pandas dataframe and a key column, and returns a dictionary
    with the key column as the dictionary keys and the rest of the columns as the values.
    """
    # Create an empty dictionary to hold the key-value pairs
    my_dict = {}
    
    # Loop through each row in the dataframe
    for index, row in df.iterrows():
        # Use the value in the key column as the dictionary key
        key_value = row[key_col]
        
        # Use the rest of the columns as the dictionary values
        value_dict = row.drop(key_col).to_dict()
        
        # Add the key-value pair to the dictionary
        my_dict[key_value] = value_dict
    
    return my_dict

## A. Generation of Metabolites dataset
We start by creating a list of all the metabolites included in the reactions of our reconstruction (1). Then we create a dataset containing all the metabolites info from Recon3D, iCHO2291 and iCHO1766 models, including supplementary information from Recon 3D (2). Now we can map back this information into the metabolites from our reconstruction and generate an excell file for uploading into Google Sheets (3).

### 1. Retrieve a list of all the metabolites from our reconstruction
The list of all the reactions and the metabolites involved are in the Rxns Sheet in the Google Sheet.

In [3]:
# give service account details to gspread
sa = gspread.service_account(filename='credentials.json')

# sa is a gspread client, which can be used for connecting to the sheets
# by using the open method and the sheet name.
cho_recon = sa.open('CHO Network Reconstruction')

# we also need to specify the page name before getting the data. In this case we use the Rxns sheet.
rxns_sheet = cho_recon.worksheet('Rxns')

In [4]:
# We can extract the data using the get_all_records method and create a pd DataFrame
df = pd.DataFrame(rxns_sheet.get_all_records())
df

,Index,Curated,Reaction,Reaction Name,Reaction Formula,Subsystem,GPR_hef,GPR_fou,GPR_yeo,GPR_Recon3D,GPR_final,GPR_Final_Thanasis,Conf. Score,Curation Notes,References
0,0,PD,10FTHF5GLUtl,"5-glutamyl-10FTHF transport, lysosomal",10fthf5glu_c --> 10fthf5glu_l,"TRANSPORT, LYSOSOMAL",,,,,,,1,No information available in the literature abo...,
1,1,PD,10FTHF5GLUtm,"5-glutamyl-10FTHF transport, mitochondrial",10fthf5glu_m --> 10fthf5glu_c,"TRANSPORT, MITOCHONDRIAL",,,,,,,1,No information available in the literature abo...,
2,2,PD,10FTHF6GLUtl,"6-glutamyl-10FTHF transport, lysosomal",10fthf6glu_c --> 10fthf6glu_l,"TRANSPORT, LYSOSOMAL",,,,,,,1,No information available in the literature abo...,
3,3,PD,10FTHF6GLUtm,"6-glutamyl-10FTHF transport, mitochondrial",10fthf6glu_m --> 10fthf6glu_c,"TRANSPORT, MITOCHONDRIAL",,,,,,,1,No information available in the literature abo...,
4,4,PD,10FTHF7GLUtl,"7-glutamyl-10FTHF transport, lysosomal",10fthf7glu_c --> 10fthf7glu_l,"TRANSPORT, LYSOSOMAL",,,,,,,1,No information available in the literature abo...,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8193,8197,,r2537,Utilized transport,lnlncgcoa_c <=> lnlncgcoa_r,Transport,,,,,,,,,
8194,8198,,r2538,Utilized transport,dlnlcgcoa_c <=> dlnlcgcoa_r,Transport,,,,,,,,,
8195,8199,,r2539,Postulated transport reaction,L2aadp6sa_c + L2aadp_m <=> L2aadp6sa_m + L2aadp_c,Transport,,,,,,,,,
8196,8200,PD,ALLTTtm,"Allantoate transport via diffusion, mitochondria",alltt_c <=> alltt_m,"Transport, mitochondria",,,,,,,1,The transport of Allantoate from the cytoplasm...,


In [5]:
# Create a cobra model to identify the metabolites involved in our reconstruction
model = cobra.Model("iCHOxxxx")
lr = []

for _, row in df.iterrows():
    r = cobra.Reaction(row['Reaction'])
    lr.append(r)
    
model.add_reactions(lr)
model

Name,iCHOxxxx
Memory address,152b96830
Number of metabolites,0
Number of reactions,8198
Number of genes,0
Number of groups,0
Objective expression,0
Compartments,


In [6]:
# With the built in function "build_reaction_from_string" we can identify the metabolites
for i,r in enumerate(tqdm(model.reactions)):
    r.build_reaction_from_string(df['Reaction Formula'][i])
    print(r)

model

  0%|          | 0/8198 [00:00<?, ?it/s]

unknown metabolite '10fthf5glu_c' created
unknown metabolite '10fthf5glu_l' created
10FTHF5GLUtl: 10fthf5glu_c --> 10fthf5glu_l
unknown metabolite '10fthf5glu_m' created
10FTHF5GLUtm: 10fthf5glu_m --> 10fthf5glu_c
unknown metabolite '10fthf6glu_c' created
unknown metabolite '10fthf6glu_l' created
10FTHF6GLUtl: 10fthf6glu_c --> 10fthf6glu_l
unknown metabolite '10fthf6glu_m' created
10FTHF6GLUtm: 10fthf6glu_m --> 10fthf6glu_c
unknown metabolite '10fthf7glu_c' created
unknown metabolite '10fthf7glu_l' created
10FTHF7GLUtl: 10fthf7glu_c --> 10fthf7glu_l
unknown metabolite '10fthf7glu_m' created
10FTHF7GLUtm: 10fthf7glu_m --> 10fthf7glu_c
unknown metabolite '10fthf_c' created
unknown metabolite '10fthf_l' created
10FTHFtl: 10fthf_c <=> 10fthf_l
unknown metabolite '10fthf_m' created
10FTHFtm: 10fthf_c <=> 10fthf_m
unknown metabolite '11docrtsl_c' created
unknown metabolite '11docrtsl_m' created
11DOCRTSLtm: 11docrtsl_c <=> 11docrtsl_m
unknown metabolite '11docrtsl_r' created
11DOCRTSLtr: 11d

ABUTt2r: 4abut_e + h_e <=> 4abut_c + h_c
unknown metabolite '4abut_l' created
unknown metabolite 'h_l' created
ABUTt2rL: 4abut_l + h_l <=> 4abut_c + h_c
ABUTt4_2_r: 4abut_e + 2.0 na1_e <=> 4abut_c + 2.0 na1_c
unknown metabolite 'occoa_m' created
ACACT100m: 3odcoa_m + coa_m --> accoa_m + occoa_m
unknown metabolite 'c81_5Zcoa_m' created
ACACT102n3m: 3oc101_7Zcoa_m + coa_m --> accoa_m + c81_5Zcoa_m
unknown metabolite 'coa_x' created
unknown metabolite 'accoa_x' created
unknown metabolite 'c81_5Zcoa_x' created
ACACT102n3p: 3oc101_7Zcoa_x + coa_x --> accoa_x + c81_5Zcoa_x
unknown metabolite '2maacoa_m' created
unknown metabolite 'ppcoa_m' created
ACACT10m: 2maacoa_m + coa_m <=> accoa_m + ppcoa_m
unknown metabolite 'nncoa_m' created
ACACT110m: 3oundcoa_m + coa_m --> accoa_m + nncoa_m
unknown metabolite 'dcacoa_m' created
ACACT120m: 3oddcoa_m + coa_m --> accoa_m + dcacoa_m
unknown metabolite 'tdec4ecoa_m' created
ACACT121n6em: 3oc121_6Ecoa_m + coa_m --> accoa_m + tdec4ecoa_m
unknown metabolit

ADNK3: adn_c + gtp_c --> amp_c + gdp_c + h_c
unknown metabolite 'dgtp_c' created
unknown metabolite 'dgdp_c' created
ADNK4: adn_c + dgtp_c --> amp_c + dgdp_c + h_c
ADNt: adn_e <=> adn_c
ADNt4: adn_e + na1_e --> adn_c + na1_c
ADNt5: adn_e + 2.0 na1_e --> adn_c + 2.0 na1_c
ADNtl: adn_l <=> adn_c
ADNtm: adn_c <=> adn_m
unknown metabolite 'adocbl_m' created
unknown metabolite 'adocbl_c' created
ADOCBLtm: adocbl_m --> adocbl_c
unknown metabolite 'adpac_c' created
unknown metabolite 'adpcoa_c' created
ADPACDAc: adpac_c + atp_c + coa_c --> adpcoa_c + amp_c + ppi_c
unknown metabolite 'adpac_x' created
ADPACTD: adpac_x <=> adpac_c
unknown metabolite 'adpcoa_x' created
unknown metabolite 'crn_x' created
unknown metabolite 'c6dc_x' created
ADPCOACROT: adpcoa_x + crn_x <=> c6dc_x + coa_x
ADPCOAPTE: adpcoa_x + h2o_x --> adpac_x + coa_x + h_x
unknown metabolite 'adpglc_c' created
unknown metabolite 'g1p_c' created
ADPGLC: adpglc_c + h2o_c --> amp_c + g1p_c + 2.0 h_c
unknown metabolite 'adpman_c' cre

ALAt4: ala_L_e + na1_e --> ala_L_c + na1_c
ALAtN1: ala_L_e + h_c + 2.0 na1_e <=> ala_L_c + h_e + 2.0 na1_c
unknown metabolite 'arg_L_c' created
unknown metabolite 'arg_L_e' created
ALAyLATthc: ala_L_e + arg_L_c + h_e --> ala_L_c + arg_L_e + h_c
unknown metabolite 'meoh_c' created
unknown metabolite 'fald_c' created
ALCD1: meoh_c + nad_c <=> fald_c + h_c + nadh_c
unknown metabolite '12ppd_R_c' created
unknown metabolite 'lald_D_c' created
ALCD21_D: 12ppd_R_c + nad_c --> h_c + lald_D_c + nadh_c
unknown metabolite '12ppd_S_c' created
unknown metabolite 'lald_L_c' created
ALCD21_L: 12ppd_S_c + nad_c --> h_c + lald_L_c + nadh_c
ALCD22_D: lald_D_c + nad_c --> h_c + mthgxl_c + nadh_c
ALCD22_L: lald_L_c + nad_c --> h_c + mthgxl_c + nadh_c
unknown metabolite 'etoh_c' created
ALCD2if: etoh_c + nad_c --> acald_c + h_c + nadh_c
ALCD2yf: etoh_c + nadp_c --> acald_c + h_c + nadph_c
unknown metabolite 'pacald_c' created
unknown metabolite 'pac_c' created
ALDD19x: h2o_c + nad_c + pacald_c <=> 2.0 h_c 

...

r1842: asn_L_e + na1_e + trp_L_c --> asn_L_c + na1_c + trp_L_e
r1843: asn_L_e + na1_e + phe_L_c --> asn_L_c + na1_c + phe_L_e
r1844: asn_L_e + na1_e + tyr_L_c --> asn_L_c + na1_c + tyr_L_e
r1846: asn_L_e + leu_L_c + na1_e --> asn_L_c + leu_L_e + na1_c
r1847: asn_L_e + na1_e + pro_L_c --> asn_L_c + na1_c + pro_L_e
r1848: asn_L_e + na1_e + val_L_c --> asn_L_c + na1_c + val_L_e
r1850: asn_L_e + hom_L_c + na1_e --> asn_L_c + hom_L_e + na1_c
r1851: asn_L_e + ile_L_c + na1_e --> asn_L_c + ile_L_e + na1_c
r1852: gly_c + na1_e + val_L_e --> gly_e + na1_c + val_L_c
r1853: ala_L_c + na1_e + val_L_e --> ala_L_e + na1_c + val_L_c
r1854: gln_L_c + na1_e + val_L_e --> gln_L_e + na1_c + val_L_c
r1855: na1_e + ser_L_c + val_L_e --> na1_c + ser_L_e + val_L_c
r1856: met_L_c + na1_e + val_L_e --> met_L_e + na1_c + val_L_c
r1857: na1_e + trp_L_c + val_L_e --> na1_c + trp_L_e + val_L_c
r1858: na1_e + phe_L_c + val_L_e --> na1_c + phe_L_e + val_L_c
r1859: na1_e + tyr_L_c + val_L_e --> na1_c + tyr_L_e + val_

r1984: hom_L_e + na1_e + orn_c --> hom_L_c + na1_c + orn_e
r1985: his_L_c + hom_L_e + na1_e --> his_L_e + hom_L_c + na1_c
r1986: citr_L_c + hom_L_e + na1_e --> citr_L_e + hom_L_c + na1_c
r1987: ile_L_e + lys_L_c + na1_e --> ile_L_c + lys_L_e + na1_c
r1988: arg_L_c + ile_L_e + na1_e --> arg_L_e + ile_L_c + na1_c
r1989: ile_L_e + na1_e + orn_c --> ile_L_c + na1_c + orn_e
r1990: his_L_c + ile_L_e + na1_e --> his_L_e + ile_L_c + na1_c
r1991: citr_L_c + ile_L_e + na1_e --> citr_L_e + ile_L_c + na1_c
r1992: gly_c + lys_L_e --> gly_e + lys_L_c
r1993: ala_L_c + lys_L_e --> ala_L_e + lys_L_c
r1994: gln_L_c + lys_L_e --> gln_L_e + lys_L_c
r1995: lys_L_e + ser_L_c --> lys_L_c + ser_L_e
r1996: lys_L_e + met_L_c --> lys_L_c + met_L_e
r1997: lys_L_e + trp_L_c --> lys_L_c + trp_L_e
r1998: lys_L_e + phe_L_c --> lys_L_c + phe_L_e
r1999: lys_L_e + tyr_L_c --> lys_L_c + tyr_L_e
r2000: cys_L_c + lys_L_e --> cys_L_e + lys_L_c
r2001: leu_L_c + lys_L_e --> leu_L_e + lys_L_c
r2002: lys_L_e + pro_L_c --> lys_L

unknown metabolite 'HC02180_c' created
r2200: HC02180_e + gthrd_c + hco3_c <=> HC02180_c + gthrd_e + hco3_e
r2201: gthrd_c + hco3_c + leuktrE4_e <=> gthrd_e + hco3_e + leuktrE4_c
r2202: bilglcur_e + gthrd_c + hco3_c <=> bilglcur_c + gthrd_e + hco3_e
r2203: HC02199_c + bilirub_e + hco3_c <=> HC02199_e + bilirub_c + hco3_e
r2204: HC02200_c + bilirub_e + hco3_c <=> HC02200_e + bilirub_c + hco3_e
r2205: HC02201_c + bilirub_e + hco3_c <=> HC02201_e + bilirub_c + hco3_e
r2206: HC02199_c + hco3_c + thyox_L_e <=> HC02199_e + hco3_e + thyox_L_c
r2207: HC02200_c + hco3_c + thyox_L_e <=> HC02200_e + hco3_e + thyox_L_c
r2208: HC02201_c + hco3_c + thyox_L_e <=> HC02201_e + hco3_e + thyox_L_c
r2209: HC02199_c + hco3_c + triodthy_e <=> HC02199_e + hco3_e + triodthy_c
r2210: HC02200_c + hco3_c + triodthy_e <=> HC02200_e + hco3_e + triodthy_c
r2211: HC02201_c + hco3_c + triodthy_e <=> HC02201_e + hco3_e + triodthy_c
r2212: HC02187_e + HC02199_c + hco3_c <=> HC02187_c + HC02199_e + hco3_e
r2213: HC02187

r2323: HC02206_e + gthrd_c + hco3_c <=> HC02206_c + gthrd_e + hco3_e
r2324: HC02207_e + gthrd_c + hco3_c <=> HC02207_c + gthrd_e + hco3_e
r2325: HC02208_e + gthrd_c + hco3_c <=> HC02208_c + gthrd_e + hco3_e
r2326: gthrd_c + hco3_c + prostgd2_e <=> gthrd_e + hco3_e + prostgd2_c
r2327: HC02210_e + gthrd_c + hco3_c <=> HC02210_c + gthrd_e + hco3_e
r2328: gthrd_c + hco3_c + prostge1_e <=> gthrd_e + hco3_e + prostge1_c
r2329: HC02213_e + gthrd_c + hco3_c <=> HC02213_c + gthrd_e + hco3_e
r2330: HC02214_e + gthrd_c + hco3_c <=> HC02214_c + gthrd_e + hco3_e
r2331: gthrd_c + hco3_c + prostgf2_e <=> gthrd_e + hco3_e + prostgf2_c
r2332: HC02216_e + gthrd_c + hco3_c <=> HC02216_c + gthrd_e + hco3_e
r2333: HC02217_e + gthrd_c + hco3_c <=> HC02217_c + gthrd_e + hco3_e
r2334: gthrd_c + hco3_c + prostgh2_e <=> gthrd_e + hco3_e + prostgh2_c
r2335: gthrd_c + hco3_c + prostgi2_e <=> gthrd_e + hco3_e + prostgi2_c
r2338: ach_e <=> ach_c
r2341: leuktrC4_e <=> leuktrC4_c
r2342: estrones_e <=> estrones_c
r234

Name,iCHOxxxx
Memory address,152b96830
Number of metabolites,5455
Number of reactions,8198
Number of genes,0
Number of groups,0
Objective expression,0
Compartments,


In [7]:
# We first create a list of the metabolites and then a pandas df with it
metabolites_list = []
for met in model.metabolites:
    metabolites_list.append(met.id)
    
metabolites = pd.DataFrame(metabolites_list, columns =['BiGG ID'])
metabolites

,BiGG ID
0,10fthf5glu_c
1,10fthf5glu_l
2,10fthf5glu_m
3,10fthf6glu_c
4,10fthf6glu_l
...,...
5450,HC02214_c
5451,HC02216_c
5452,HC02217_c
5453,fdxrd_c


### 2. Retrieve information from all the metabolites on Recon3D, iCHO2291 and iCHO1766
We use two datasets for this, first we take information from the Recon3D.xml, iCHO2291.xml and iCHO1766 files from which we get the metabolite ID, Name, Formula and Compartment. We then add the metadata for the available metabolites from Recon3D supplementary files.

In [8]:
# read the Recon3D model
recon3d_model = read_sbml_model('../Data/GPR_Curation/Recon3D.xml')

In [9]:
# Generate a dataset containing all the metabolites, chemical formula of each metabolite and compartment
num_rows = len(recon3d_model.metabolites)
recon3d_model_metabolites = pd.DataFrame(index=range(num_rows), columns=['BiGG ID', 'Name', 'Formula', 'Compartment'])
for i,met in enumerate(recon3d_model.metabolites):
    id_ = met.id
    name = met.name
    formula = met.formula
    comp = met.compartment
    recon3d_model_metabolites.iloc[i] = [id_, name, formula, comp]

In [10]:
recon3d_model_metabolites

,BiGG ID,Name,Formula,Compartment
0,10fthf_c,10-Formyltetrahydrofolate,C20H21N7O7,c
1,10fthf_l,10-Formyltetrahydrofolate,C20H21N7O7,l
2,10fthf_m,10-Formyltetrahydrofolate,C20H21N7O7,m
3,11docrtsl_c,11docrtsl c,C21H30O4,c
4,11docrtsl_m,11docrtsl c,C21H30O4,m
...,...,...,...,...
5830,caproic_e,Caproic Acid,C6H11O2,e
5831,1a25dhvitd2_c,"1-alpha,25-Dihydroxyvitamin D2",C28H44O3,c
5832,1a25dhvitd2_e,"1-alpha,25-Dihydroxyvitamin D2",C28H44O3,e
5833,protein_c,Torasemide-M3,None,c


In [12]:
# read the Yeo's model
iCHO2291_model = read_sbml_model('../Data/Reconciliation/models/iCHO2291.xml')

In [40]:
# Generate a dataset containing all the metabolites, chemical formula of each metabolite and compartment from Yeo's model
num_rows = len(iCHO2291_model.metabolites)
iCHO2291_model_metabolites = pd.DataFrame(index=range(num_rows), columns=['BiGG ID', 'Name', 'Formula', 'Compartment'])
for i,met in enumerate(iCHO2291_model.metabolites):
    id_ = met.id
    name = met.name
    formula = met.formula
    comp = met.compartment
    iCHO2291_model_metabolites.iloc[i] = [id_, name, formula, comp]
    
iCHO2291_model_metabolites['BiGG ID'] = iCHO2291_model_metabolites['BiGG ID'].str.replace("[", "_", regex=False)
iCHO2291_model_metabolites['BiGG ID'] = iCHO2291_model_metabolites['BiGG ID'].str.replace("]", "", regex=False)
iCHO2291_model_metabolites

,BiGG ID,Name,Formula,Compartment
0,10fthf5glu_c,10-formyltetrahydrofolate-[Glu](5),C40H45N11O19,c
1,10fthf5glu_l,10-formyltetrahydrofolate-[Glu](5),C40H45N11O19,l
2,10fthf5glu_m,10-formyltetrahydrofolate-[Glu](5),C40H45N11O19,m
3,10fthf6glu_c,10-formyltetrahydrofolate-[Glu](6),C45H51N12O22,c
4,10fthf6glu_l,10-formyltetrahydrofolate-[Glu](6),C45H51N12O22,l
...,...,...,...,...
3967,Rtotal3crn_c,Rtotal3crn[c],CO2R3C7H14NO2,c
3968,Rtotal3crn_m,Rtotal3crn[m],CO2R3C7H14NO2,m
3969,Rtotalcrn_c,Rtotalcrn[c],CO2RC7H14NO2,c
3970,Rtotalcrn_m,Rtotalcrn[m],CO2RC7H14NO2,m


In [14]:
# read Hefzi's model
iCHO1766_model = read_sbml_model('../Data/Reconciliation/models/iCHOv1_final.xml')

In [15]:
# Generate a dataset containing all the metabolites, chemical formula of each metabolite and compartment from Hefzi's model
num_rows = len(iCHO1766_model.metabolites)
iCHO1766_model_metabolites = pd.DataFrame(index=range(num_rows), columns=['BiGG ID', 'Name', 'Formula', 'Compartment'])
for i,met in enumerate(iCHO1766_model.metabolites):
    id_ = met.id
    name = met.name
    formula = met.formula
    comp = met.compartment
    iCHO1766_model_metabolites.iloc[i] = [id_, name, formula, comp]

iCHO1766_model_metabolites

,BiGG ID,Name,Formula,Compartment
0,coke_r,cocaine,C17H21NO4,r
1,h2o_r,H2O,H2O,r
2,bz_r,benzoate,C7H5O2,r
3,egme_r,ecgonine methyl ester,C10H17NO3,r
4,h_r,proton,H,r
...,...,...,...,...
4451,igg_g,igg[g],None,g
4452,nicrns_c,Nicotinate D-ribonucleoside,C11H13NO6,c
4453,bilglcur_r,Bilirubin monoglucuronide,C39H44N4O12,r
4454,pcollglys_c,Procollagen L-lysine,C7H14N3O2R2,c


In [16]:
models_metabolites = pd.concat([recon3d_model_metabolites, iCHO2291_model_metabolites, iCHO1766_model_metabolites])
models_metabolites = models_metabolites.groupby('BiGG ID').first()
models_metabolites = models_metabolites.reset_index(drop = False)
models_metabolites

,BiGG ID,Name,Formula,Compartment
0,10fthf5glu_c,10-formyltetrahydrofolate-[Glu](5),C40H45N11O19,c
1,10fthf5glu_e,10-formyltetrahydrofolate-[Glu](5),C40H45N11O19,e
2,10fthf5glu_l,10-formyltetrahydrofolate-[Glu](5),C40H45N11O19,l
3,10fthf5glu_m,10-formyltetrahydrofolate-[Glu](5),C40H45N11O19,m
4,10fthf6glu_c,10-formyltetrahydrofolate-[Glu](6),C45H51N12O22,c
...,...,...,...,...
8527,zym_int2_r,Zymosterol intermediate 2 C27H42O,C27H42O,r
8528,zymst_c,Zymosterol C27H44O,C27H44O,c
8529,zymst_r,Zymosterol C27H44O,C27H44O,r
8530,zymstnl_c,5alpha-cholest-8-en-3beta-ol,C27H46O,c


In [17]:
#Generation of a dataset containing all the information from Recon3D metabolites Supplementary Data.
recon3d_metabolites_meta = pd.read_excel('../Data/Metabolites/metabolites.recon3d.xlsx', header = 0)
recon3d_metabolites_meta['BiGG ID'] = recon3d_metabolites_meta['BiGG ID'].str.replace("[", "_", regex=False)
recon3d_metabolites_meta['BiGG ID'] = recon3d_metabolites_meta['BiGG ID'].str.replace("]", "", regex=False)
recon3d_metabolites_meta

,BiGG ID,KEGG,CHEBI,PubChem,Inchi,Hepatonet,EHMNID,SMILES,INCHI2,CC_ID,Stereoisomer Information of Metabolite Identified,Charge of the Metabolite Identified,CID_ID,PDB (ligand-expo) Experimental Coordinates File Url,Pub Chem Url,ChEBI Url
0,10fthf5glu_c,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10fthf5glu_l,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10fthf5glu_m,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10fthf6glu_c,NaN,NaN,NaN,InChI=1/C45H58N12O22/c46-45-55-36-35(38(67)56-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10fthf6glu_l,NaN,NaN,NaN,InChI=1/C45H58N12O22/c46-45-55-36-35(38(67)56-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5282,NaN,NaN,CID100015232,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5283,NaN,NaN,CID100123634,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5284,NaN,NaN,CID100206527,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5285,NaN,NaN,CID105479141,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
# Transformation of the "recon3d_metabolites_meta" into a dict to map it into the "recon3d_model_metabolites"
recon3dmet_dict = df_to_dict(recon3d_metabolites_meta, 'BiGG ID')

In [19]:
# Mapping into the "recon3d_model_metabolites" dataset
models_metabolites[['KEGG','CHEBI', 'PubChem','Inchi', 'Hepatonet', 'EHMNID', 'SMILES', 'INCHI2',
                          'CC_ID','Stereoisomer Information of Metabolite Identified', 'Charge of the Metabolite Identified',
    'CID_ID','PDB (ligand-expo) Experimental Coordinates  File Url', 'Pub Chem Url',
    'ChEBI Url']] = models_metabolites['BiGG ID'].apply(lambda x: pd.Series(recon3dmet_dict.get(x, None), dtype=object))

In [20]:
models_metabolites

,BiGG ID,Name,Formula,Compartment,KEGG,CHEBI,PubChem,Inchi,Hepatonet,EHMNID,SMILES,INCHI2,CC_ID,Stereoisomer Information of Metabolite Identified,Charge of the Metabolite Identified,CID_ID,PDB (ligand-expo) Experimental Coordinates File Url,Pub Chem Url,ChEBI Url
0,10fthf5glu_c,10-formyltetrahydrofolate-[Glu](5),C40H45N11O19,c,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10fthf5glu_e,10-formyltetrahydrofolate-[Glu](5),C40H45N11O19,e,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10fthf5glu_l,10-formyltetrahydrofolate-[Glu](5),C40H45N11O19,l,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10fthf5glu_m,10-formyltetrahydrofolate-[Glu](5),C40H45N11O19,m,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10fthf6glu_c,10-formyltetrahydrofolate-[Glu](6),C45H51N12O22,c,NaN,NaN,NaN,InChI=1/C45H58N12O22/c46-45-55-36-35(38(67)56-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8527,zym_int2_r,Zymosterol intermediate 2 C27H42O,C27H42O,r,C05437,18252.0,22298942.0,InChI=1S/C27H42O/c1-18(2)7-6-8-19(3)23-11-12-2...,NaN,NaN,[H][C@@]12CCC3=C(CC[C@]4(C)[C@]([H])(CC[C@@]34...,InChI=1S/C27H42O/c1-18(2)7-6-8-19(3)23-11-12-2...,NaN,NaN,Neutral,22298942.0,NaN,https://pubchem.ncbi.nlm.nih.gov/compound/2229...,https://www.ebi.ac.uk/chebi/searchId.do?chebiI...
8528,zymst_c,Zymosterol C27H44O,C27H44O,c,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8529,zymst_r,Zymosterol C27H44O,C27H44O,r,C05437,18252.0,92746.0,InChI=1S/C27H44O/c1-18(2)7-6-8-19(3)23-11-12-2...,HC01451,C05437,[H][C@@]12CCC3=C(CC[C@]4(C)[C@]([H])(CC[C@@]34...,InChI=1S/C27H44O/c1-18(2)7-6-8-19(3)23-11-12-2...,NaN,NaN,Neutral,92746.0,NaN,https://pubchem.ncbi.nlm.nih.gov/compound/92746,https://www.ebi.ac.uk/chebi/searchId.do?chebiI...
8530,zymstnl_c,5alpha-cholest-8-en-3beta-ol,C27H46O,c,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
# Transform the final Recon3D Metabolites dataset into a dictionary to map it into our dataset
final_met_dict = df_to_dict(models_metabolites, 'BiGG ID')

### 3. Add the information from Recon3D in our metabolites dataset
With the dictionary created in **Step 2** we can use the information to map it in the metabolites dataset created in **Step 1** which contains all the metabolites of our reconstruction.

In [ ]:
metabolites[['Name', 'Formula', 'Compartment', 'KEGG','CHEBI', 'PubChem','Inchi', 'Hepatonet', 'EHMNID', 'SMILES',
             'INCHI2','CC_ID','Stereoisomer Information of Metabolite Identified', 'Charge of the Metabolite Identified',
    'CID_ID','PDB (ligand-expo) Experimental Coordinates  File Url', 'Pub Chem Url',
    'ChEBI Url']] = metabolites['BiGG ID'].apply(lambda x: pd.Series(final_met_dict.get(x, None), dtype=object))

In [ ]:
# Update the Compartment column in the final dataset
for i,row in metabolites.iterrows():
    if row['Compartment'] == 'c':
        metabolites.loc[i, 'Compartment'] = 'c - cytosol'
    if row['Compartment'] == 'l':
        metabolites.loc[i, 'Compartment'] = 'l - lysosome'
    if row['Compartment'] == 'm':
        metabolites.loc[i, 'Compartment'] = 'm - mitochondria'
    if row['Compartment'] == 'r':
        metabolites.loc[i, 'Compartment'] = 'r - endoplasmic reticulum'
    if row['Compartment'] == 'e':
        metabolites.loc[i, 'Compartment'] = 'e - extracellular space'
    if row['Compartment'] == 'x':
        metabolites.loc[i, 'Compartment'] = 'x - peroxisome/glyoxysome'
    if row['Compartment'] == 'n':
        metabolites.loc[i, 'Compartment'] = 'n - nucleus'
    if row['Compartment'] == 'g':
        metabolites.loc[i, 'Compartment'] = 'g - golgi apparatus'
    if row['Compartment'] == 'im':
        metabolites.loc[i, 'Compartment'] = 'im - intermembrane space of mitochondria'

In [ ]:
metabolites.to_excel('../Data/Metabolites/metabolites.xlsx')

### 4. Unique metabolite identification

In [ ]:
# give service account details to gspread
sa = gspread.service_account(filename='credentials.json')

# sa is a gspread client, which can be used for connecting to the sheets
# by using the open method and the sheet name.
cho_recon = sa.open('CHO Network Reconstruction')

# we also need to specify the page name before getting the data. In this case we use the Rxns sheet.
metabolites_sheet = cho_recon.worksheet('Metabolites')

# We can extract the data using the get_all_records method and create a pd DataFrame
metabolites = pd.DataFrame(metabolites_sheet.get_all_records())
metabolites = metabolites.set_index('Index')
metabolites

In [ ]:
print("Duplicated rxns by BiGG ID = ", len(metabolites['BiGG ID']) - len(metabolites['BiGG ID'].unique()))
print("Duplicated rxns by Name = ", len(metabolites['Name']) - len(metabolites['Name'].unique()))
print("Duplicated rxns by Formula = ", len(metabolites['Formula']) - len(metabolites['Formula'].unique()))
print("Duplicated rxns by KEGG = ", len(metabolites['KEGG']) - len(metabolites['KEGG'].unique()))

In [ ]:
# Group the metabolites by formula and create an excel file to curate them manually.
grouped_by_formula = metabolites.groupby('Formula', group_keys=True).apply(lambda x:x)
grouped_by_formula.to_excel('../Data/Metabolites/grouped_by_formula.xlsx')

## B. Metabolites Curation
In this second part of the notebook we curate missing information in the metabolites dataset generated above. Since many metabolites have been manually curated in the "Metabolites" google sheet file, we generate a new dataframe using the gspread library to obtain the metabolites dataset with all the changes

### Metabolites Curation

In [63]:
# give service account details to gspread
sa = gspread.service_account(filename='credentials.json')

# sa is a gspread client, which can be used for connecting to the sheets
# by using the open method and the sheet name.
cho_recon = sa.open('CHO Network Reconstruction')

# we also need to specify the page name before getting the data. In this case we use the Rxns sheet.
metabolites_sheet = cho_recon.worksheet('Metabolites')

# We can extract the data using the get_all_records method and create a pd DataFrame
metabolites = pd.DataFrame(metabolites_sheet.get_all_records())
metabolites = metabolites.set_index('Index')
metabolites = metabolites.reset_index(drop=True)
metabolites

,Curated,BiGG ID,Name,Formula,Compartment,KEGG,CHEBI,PubChem,Inchi,Hepatonet,EHMNID,SMILES,INCHI2,CC_ID,Stereoisomer Information of Metabolite Identified,Charge of the Metabolite Identified,CID_ID,PDB (ligand-expo) Experimental Coordinates File Url,Pub Chem Url,ChEBI Url
0,,10fthf5glu_c,10-formyltetrahydrofolate-[Glu](5),C40H45N11O19,c - cytosol,,,,,,,,,,,,,,,
1,,10fthf5glu_l,10-formyltetrahydrofolate-[Glu](5),C40H45N11O19,l - lysosome,,,,,,,,,,,,,,,
2,,10fthf5glu_m,10-formyltetrahydrofolate-[Glu](5),C40H45N11O19,m - mitochondria,,,,,,,,,,,,,,,
3,,10fthf6glu_c,10-formyltetrahydrofolate-[Glu](6),C45H51N12O22,c - cytosol,,,,InChI=1/C45H58N12O22/c46-45-55-36-35(38(67)56-...,,,,,,,,,,,
4,,10fthf6glu_l,10-formyltetrahydrofolate-[Glu](6),C45H51N12O22,l - lysosome,,,,InChI=1/C45H58N12O22/c46-45-55-36-35(38(67)56-...,,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5450,,HC02216_c,Prostaglandin-f2beta,C20H33O5,c - cytosol,,28922,439702,InChI=1S/C20H34O5/c1-2-3-6-9-15(21)12-13-17-16...,,,CCCCC[C@H](O)\C=C\[C@H]1[C@H](O)C[C@@H](O)[C@@...,InChI=1S/C20H34O5/c1-2-3-6-9-15(21)12-13-17-16...,,,Neutral,5280506,,https://pubchem.ncbi.nlm.nih.gov/compound/5280506,https://www.ebi.ac.uk/chebi/searchId.do;jsessi...
5451,,HC01361_e,Dihydroneopterin,C9H13N5O4,e - extracellular space,C04874,17001,,InChI=1S/C9H13N5O4/c10-9-13-7-5(8(18)14-9)12-3...,,,Nc1nc2c(c(=O)[nH]1)N=C(C(O)C(O)CO)CN2,,,,,,,,
5452,,HC02217_c,Prostaglandin-g2,C20H31O6,c - cytosol,,27647,,InChI=1S/C20H32O6/c1-2-3-6-9-15(24-23)12-13-17...,,C05956,CCCCC[C@H](OO)\C=C\[C@H]1[C@H]2C[C@H](OO2)[C@@...,InChI=1S/C20H32O6/c1-2-3-6-9-15(24-23)12-13-17...,PGX,,Neutral,5280883,http://ligand-expo.rcsb.org/pyapps/ldHandler.p...,https://pubchem.ncbi.nlm.nih.gov/compound/5280883,https://www.ebi.ac.uk/chebi/searchId.do?chebiI...
5453,PD,fdxo_2_2_c,Oxidized ferredoxin,Fe2S2X,c - cytosol,C00139,17908,,,,,[*][*][Fe+3]1([*][*])[S-2][Fe+3]([*][*])([*][*...,,,,,,,,


In [66]:
# Get BiGG descriptive names
import requests
from bs4 import BeautifulSoup
import time

# Unknown Mets: metabolites without names
unkown_mets = metabolites[metabolites['Name'] == '']

Descriptive_Names = [''] * len(unkown_mets)
Formulae = [''] * len(Descriptive_Names)
Changed = [True] * len(Descriptive_Names)

for Met_Counter, metID in enumerate(tqdm(unkown_mets['BiGG ID'].iloc[:])):
    print(Met_Counter)
    input_str = metID[:-2]
    response = requests.get(f"http://bigg.ucsd.edu/universal/metabolites/{input_str}")
    time.sleep(1)
    # Check if the request was successful
    if response.status_code != 200:
        D_Name = "BiGG ID not found in BiGG"
        Formulae_B = "BiGG ID not found in BiGG"
        Changed[Met_Counter] = False       
    else:    
        soup = BeautifulSoup(response.content, 'html.parser')
        N_Header = soup.find('h4', string='Descriptive name:')
        D_Name = N_Header.find_next_sibling('p').text
        N_Formulae = soup.find('h4', string='Formulae in BiGG models: ')
        Formulae_B = N_Formulae.find_next_sibling('p').text    
        if D_Name is None:
            D_Name = "Name not found in BiGG"            
        elif Formulae_B is None:
            Formulae_B = "Formula not found in BiGG"                
    Descriptive_Names[Met_Counter] = D_Name
    Formulae[Met_Counter] = Formulae_B

  0%|          | 0/160 [00:00<?, ?it/s]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159


### Update empty metabolites

In [67]:
for Met_Counter, metID in enumerate(unkown_mets['BiGG ID']):
    print('before',unkown_mets['BiGG ID'].iloc[Met_Counter])
    print('before',unkown_mets['Formula'].iloc[Met_Counter])
    print('before',unkown_mets['Name'].iloc[Met_Counter])
    if unkown_mets['Formula'].iloc[Met_Counter] == '':
        unkown_mets['Formula'].iloc[Met_Counter] = Formulae[Met_Counter]  
    unkown_mets['Name'].iloc[Met_Counter] = Descriptive_Names[Met_Counter]
    print('..............................................')
    print('after',unkown_mets['BiGG ID'].iloc[Met_Counter])
    print('after',unkown_mets['Formula'].iloc[Met_Counter])
    print('after',unkown_mets['Name'].iloc[Met_Counter])
    print('..............................................')
    print('..............................................')
    print('..............................................')

before 3hoc101_7Zcoa_m
before C31H48N7O18P3S
before 
..............................................
after 3hoc101_7Zcoa_m
after C31H48N7O18P3S
after BiGG ID not found in BiGG
..............................................
..............................................
..............................................
before 3hoc101_7Zcoa_x
before C31H48N7O18P3S
before 
..............................................
after 3hoc101_7Zcoa_x
after C31H48N7O18P3S
after BiGG ID not found in BiGG
..............................................
..............................................
..............................................
before 3hoc121_6Ecoa_m
before C33H52N7O18P3S
before 
..............................................
after 3hoc121_6Ecoa_m
after C33H52N7O18P3S
after BiGG ID not found in BiGG
..............................................
..............................................
..............................................
before 3hoc122_6Z_9Zcoa_m
before C33H50N7O18P3S
befo

/var/folders/_x/tfg8s2ks4n1ftkkwzp5sqjpc0000gn/T/ipykernel_4175/3971818088.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unkown_mets['Name'].iloc[Met_Counter] = Descriptive_Names[Met_Counter]
/var/folders/_x/tfg8s2ks4n1ftkkwzp5sqjpc0000gn/T/ipykernel_4175/3971818088.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unkown_mets['Formula'].iloc[Met_Counter] = Formulae[Met_Counter]


In [68]:
metabolites.update(unkown_mets)

# Manual Curation
for bigg_id in metabolites['BiGG ID']:
    # xtra = Xanthurenic acid; C10H6NO4
    # http://bigg.ucsd.edu/models/iCHOv1/reactions/r0647
    if 'xtra' in bigg_id:
        metabolites.loc[metabolites['BiGG ID'] == bigg_id, 'Name'] = 'Xanthurenic acid'
        metabolites.loc[metabolites['BiGG ID'] == bigg_id, 'Formula'] = 'C10H6NO4'
    # chedxch = Bilirubin-monoglucuronoside; C39H42N4O122-
    # Reactions name = 'ATP-binding Cassette (ABC) TCDB:3.A.1.208.2' --> https://metabolicatlas.org/identifier/TCDB/3.A.1.208.2
    elif 'chedxch' in bigg_id:
        metabolites.loc[metabolites['BiGG ID'] == bigg_id, 'Name'] = 'Bilirubin-monoglucuronoside'
        metabolites.loc[metabolites['BiGG ID'] == bigg_id, 'Formula'] = 'C39H42N4O122-'
    # chatGTP
    elif '3hoc246_6Z_9Z_12Z_15Z_18Z_21Zcoa' in bigg_id:
        metabolites.loc[metabolites['BiGG ID'] == bigg_id, 'Name'] = 'CoA molecule that has a 24-carbon fatty acid with six double bonds, with the location of the double bonds specified by the numbers and Zs'
    # chatGTP
    elif 'c247_2Z_6Z_9Z_12Z_15Z_18Z_21Zcoa' in bigg_id:
        metabolites.loc[metabolites['BiGG ID'] == bigg_id, 'Name'] = 'CoA molecule that has a modified version of the same 24-carbon fatty acid, with a hydroxyl group added at the third carbon position'
    # chatGTP
    elif '3hoc143_5Z_8Z_11Zcoa' in bigg_id:
        metabolites.loc[metabolites['BiGG ID'] == bigg_id, 'Name'] = 'CoA molecule that has a 14-carbon fatty acid with three double bonds, with the location of the double bonds specified by the numbers and Zs.'
    # chatGTP
    elif '3oc143_5Z_8Z_11Zcoa' in bigg_id:
        metabolites.loc[metabolites['BiGG ID'] == bigg_id, 'Name'] = 'CoA molecule that has a modified version of the same 14-carbon fatty acid, with the hydroxyl group removed and one of the double bonds converted to a keto group'
    # chatGTP
    elif 'acgalgalacglcgalgluside' in bigg_id:
        metabolites.loc[metabolites['BiGG ID'] == bigg_id, 'Name'] = 'Complex glycosphingolipid that contains multiple sugar residues'

    # 12e8hdx WTF?
    # hdxur Dead End

metabolites.to_excel('../Data/Metabolites/metabolites_final.xlsx')

/var/folders/_x/tfg8s2ks4n1ftkkwzp5sqjpc0000gn/T/ipykernel_4175/2563423044.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unkown_mets['Name'].iloc[Met_Counter] = Descriptive_Names[Met_Counter]
/var/folders/_x/tfg8s2ks4n1ftkkwzp5sqjpc0000gn/T/ipykernel_4175/2563423044.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unkown_mets['Formula'].iloc[Met_Counter] = Formulae[Met_Counter]


In [70]:
metabolites[metabolites['Name'] == '']

,Curated,BiGG ID,Name,Formula,Compartment,KEGG,CHEBI,PubChem,Inchi,Hepatonet,EHMNID,SMILES,INCHI2,CC_ID,Stereoisomer Information of Metabolite Identified,Charge of the Metabolite Identified,CID_ID,PDB (ligand-expo) Experimental Coordinates File Url,Pub Chem Url,ChEBI Url


In [35]:
metabolites[metabolites['BiGG ID'] == 'xtra']

,Curated,BiGG ID,Name,Formula,Compartment,KEGG,CHEBI,PubChem,Inchi,Hepatonet,EHMNID,SMILES,INCHI2,CC_ID,Stereoisomer Information of Metabolite Identified,Charge of the Metabolite Identified,CID_ID,PDB (ligand-expo) Experimental Coordinates File Url,Pub Chem Url,ChEBI Url
